In [3]:
import pandas as pd
import numpy as np

import speech_recognition as sr
from pydub import AudioSegment
from collections import Counter
import warnings

C:\Users\ssttg\anaconda3\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [10]:
import pickle
from Audioandtext import VoteClassifier
from Audioandtext import AudioPredictor

In [11]:
with open('text_classifier_model.pkl', 'rb') as f:
    text_classifier = pickle.load(f)

with open('audio_classifier_model.pkl', 'rb') as f:
    audio_classifier = pickle.load(f)

C:\Users\ssttg\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 24 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
C:\Users\ssttg\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [14]:
audio_path = "./TESS Toronto emotional speech set data/OAF_angry/OAF_burn_angry.wav"

In [16]:
emotion_labels_audio = ["Happiness", "Neutral", "Sadness", "Anger", "Fear", "Disgust"]
emotion_labels_text = ["Anger", "Disgust", "Fear", "Happiness", "Sadness", "Surprise", "Neutral"]


In [18]:
def extract_text_from_audio(audio_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio_data = recognizer.record(source)
    text = recognizer.recognize_google(audio_data)
    return text


In [20]:
audio_sentiment = audio_classifier.audio_classify(audio_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 317ms/step


In [21]:
audio_sentiment

'3'

In [22]:
text = extract_text_from_audio(audio_path)
pred = text_classifier.text_classify(text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


In [23]:
pred

'6'

In [24]:
emotion_labels_audio = ["Happiness", "Neutral", "Sadness", "Anger", "Fear", "Disgust"]
emotion_labels_text = ["Anger", "Disgust", "Fear", "Happiness", "Sadness", "Surprise", "Neutral"]

audio_emotion = emotion_labels_audio[int(audio_sentiment)]
text_emotion = emotion_labels_text[int(pred)]

print(audio_emotion)
print(text_emotion)

Anger
Neutral


In [ ]:
import pandas as pd
import numpy as np
import speech_recognition as sr
import pickle
from collections import Counter

# Load models
with open('text_classifier_model.pkl', 'rb') as f:
    text_classifier = pickle.load(f)

with open('audio_classifier_model.pkl', 'rb') as f:
    audio_classifier = pickle.load(f)

# Emotion labels
emotion_labels_audio = ["Happiness", "Neutral", "Sadness", "Anger", "Fear", "Disgust"]
emotion_labels_text = ["Anger", "Disgust", "Fear", "Happiness", "Sadness", "Surprise", "Neutral"]

# Function to extract text from audio
def extract_text_from_audio(audio_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio_data = recognizer.record(source)
    text = recognizer.recognize_google(audio_data)
    return text

# Path to your audio file
audio_path = "./TESS Toronto emotional speech set data/OAF_angry/OAF_burn_angry.wav"

# Get predictions from both models
audio_sentiment = audio_classifier.audio_classify(audio_path)
text = extract_text_from_audio(audio_path)
pred = text_classifier.text_classify(text)

# Map indices to emotion labels
audio_emotion = emotion_labels_audio[int(audio_sentiment)]
text_emotion = emotion_labels_text[int(pred)]

# Priority Fusion (Audio has higher priority if different)
def priority_fusion(audio_emotion, text_emotion):
    # If both models agree, return the same emotion
    if audio_emotion == text_emotion:
        return audio_emotion
    
    # If models disagree, prioritize audio model prediction
    # Apply majority voting
    predictions = [audio_emotion, text_emotion]
    counter = Counter(predictions)
    
    # If the counter finds more than one, we give priority to the audio model
    final_emotion, count = counter.most_common(1)[0]
    
    if final_emotion != audio_emotion:
        # Audio has more priority, return the audio's prediction
        return audio_emotion
    
    return final_emotion

# Get final prediction using the priority fusion
final_emotion = priority_fusion(audio_emotion, text_emotion)

# Output the results
print(f"Audio Emotion: {audio_emotion}")
print(f"Text Emotion: {text_emotion}")
print(f"Final Predicted Emotion (Priority Fusion): {final_emotion}")


In [46]:
def priority_fusion_audiotext(audio_emotion, text_emotion):
    
    if audio_emotion == text_emotion:
        return audio_emotion
    
    
    predictions = [audio_emotion, text_emotion]
    counter = Counter(predictions)
    
    
    final_emotion, count = counter.most_common(1)[0]
    
    if final_emotion != audio_emotion:
        
        return audio_emotion
    
    return final_emotion


In [42]:
# Path to your audio file
audio_path = "./AudioWAV/1001_IEO_ANG_HI.wav"

# Get predictions from both models
audio_sentiment = audio_classifier.audio_classify(audio_path)
text = extract_text_from_audio(audio_path)
pred = text_classifier.text_classify(text)

# Map indices to emotion labels
audio_emotion = emotion_labels_audio[int(audio_sentiment)]
text_emotion = emotion_labels_text[int(pred)]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 687ms/step


In [48]:
emotion_labels_audio = ["Happiness", "Neutral", "Sadness", "Anger", "Fear", "Disgust"]
emotion_labels_text = ["Anger", "Disgust", "Fear", "Happiness", "Sadness", "Surprise", "Neutral"]

final_emotion = priority_fusion_audiotext(audio_emotion, text_emotion)

# Output the results
print(f"Audio Emotion: {audio_emotion}")
print(f"Text Emotion: {text_emotion}")
print(f"Final Predicted Emotion (Priority Fusion): {final_emotion}")


Audio Emotion: Fear
Text Emotion: Neutral
Final Predicted Emotion (Priority Fusion): Fear
